## Silver Data Transformation

In [0]:
path = "abfs://bronze@stgnsahu10.dfs.core.windows.net/"
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df = spark.read.format("delta")\
    .option("header", True)\
    .option("inferschema", True)\
    .load(f"{path}netflix_titles")


In [0]:
df.display()

In [0]:
df = df.fillna({"duration_minutes": 0, "duration_seasons": 1})
df.display()

In [0]:
df = df.withColumn("duration_minutes", df["duration_minutes"].cast(IntegerType()))\
    .withColumn("duration_seasons", df["duration_seasons"].cast(IntegerType()))
df.printSchema()

In [0]:
df.display()

In [0]:
df = df.withColumn("Shorttitle", split(col('title'), ':')[0])
df.display()

In [0]:
df = df.withColumn("rating", split(col('rating'), '-')[0])
df.display()

In [0]:
df = df.withColumn("type_flag", when(col('type') == "Movie", 1)\
    .when(col('type') == "TV Show", 2)\
    .otherwise(0))
df.display()

In [0]:
# window function use
from pyspark.sql.window import Window

In [0]:
df= df.withColumn("duration_ranking", dense_rank().over(Window.orderBy(col('duration_minutes').desc())))
df.display()

In [0]:
# create view
df.createOrReplaceGlobalTempView("temp_view")

In [0]:
%sql
select * FROM global_temp.tem_view

In [0]:
df = spark.sql("""
               SELECT * FROM global_temp.temp_view
               """)
df.display()

In [0]:
# want to know how many movie and how manu tv shows are here
df_summery = df.groupBy("type_flag").count()
df_summery2 = df.groupBy("type").agg(count("*").alias("total_count"))
df_summery.display()
df_summery2.display()



Databricks visualization. Run in Databricks to view.

In [0]:
# saving cleaned data into silver path with override the data
silver_path = "abfs://silver@stgnsahu10.dfs.core.windows.net/"
df.write.format("delta")\
    .mode("overwrite")\
    .option("path", f"{silver_path}netflix_titles").save()